In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA

In [95]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [110]:
df=pd.read_csv('train.csv')

In [111]:
df

,Unnamed: 0,stock,date,neg,neu,pos,compound,volume
0,0,BAC,2023-10-01,0.0485,0.8330,0.1190,0.9958,NaN
1,1,BAC,2023-10-02,0.0485,0.8330,0.1190,0.9958,50388600.0
2,2,BAC,2023-10-03,0.0200,0.8730,0.1070,0.9932,62144500.0
3,3,BAC,2023-10-04,0.0200,0.8730,0.1070,0.9932,55696400.0
4,4,BAC,2023-10-05,0.1065,0.8100,0.0835,-0.3283,49044100.0
...,...,...,...,...,...,...,...,...
400,400,VZ,2023-10-22,0.0440,0.8595,0.0960,0.1778,30238900.0
401,401,VZ,2023-10-23,0.0255,0.8365,0.1380,0.9913,31725800.0
402,402,VZ,2023-10-24,0.0235,0.8080,0.1680,1.0000,61617900.0
403,403,VZ,2023-10-25,0.0170,0.8275,0.1555,0.9991,30832800.0


In [112]:
stocks=df.stock.unique()

In [113]:
cv = TimeSeriesSplit(5, test_size=4)    

start = 1
end = 27
ma_rmses = np.zeros((5, len(range(start, end))))


i = 0
df_tt=[]
df_ho=[]

for train_index, test_index in cv.split(df[df.stock==i]):
        tt = df.iloc[train_index]
        df_tt.append(tt)
        ho = df.iloc[test_index]
        df_ho.append(ho)
        j = 0
        for q in range(start, end):
            pred = df_tt.volume[-q:].mean() * np.ones(len(df_ho))
        
            ma_rmses[i,j] = np.sqrt(mean_squared_error(df_ho.volume.values, pred))
            j = j + 1
        i = i + 1

AttributeError: 'list' object has no attribute 'volume'